## Connect and Get Graph

In [51]:
import pypgx.pg.rdbms.graph_server as graph_server

base_url = "https://localhost:7007"
username = "graphuser"
password = "WELcome123##"

instance = graph_server.get_instance(base_url, username, password)
session = instance.create_session("jupyter")
analyst = session.create_analyst()

print(session)

PgxSession(id: 067a4438-a45c-47bf-bb14-f3932928ed59, name: python_pgx_client)


In [52]:
graph = session.get_graph("GRAPH2")
graph

PgxGraph(name: Moneyflows, v: 18000, e: 309240, directed: True, memory(Mb): 17)

## Modify Graph

In [97]:
graph = graph.clone_and_execute_pgql("""
INSERT EDGE e BETWEEN x AND y LABELS (transferred_to) PROPERTIES (e.amount = 100)
  FROM MATCH (x:account), MATCH (y:account) WHERE x.acc_id = 1 AND y.acc_id = 2
""")

In [98]:
graph

PgxGraph(name: sub-graph_42, v: 10000, e: 299244, directed: True, memory(Mb): 19)

In [102]:
graph.destroy_vertex_property_if_exists("is_fraud")
graph.create_vertex_property("boolean", "is_fraud")

VertexProperty(name: is_fraud, type: boolean, graph: sub-graph_42)

In [103]:
graph = graph.clone_and_execute_pgql("""
UPDATE x SET (x.is_fraud = true) FROM MATCH (x:account) WHERE x.acc_id = 1
""")

## Centrality

In [53]:
graph = graph.filter(pgx.EdgeFilter("edge.label()='TRANSFERRED_TO'"))
graph

PgxGraph(name: sub-graph_23, v: 10000, e: 299240, directed: True, memory(Mb): 14)

In [56]:
result = analyst.in_degree_centrality(graph)
result

VertexProperty(name: in_degree, type: integer, graph: sub-graph_23)

In [60]:
result = analyst.out_degree_centrality(graph)
result

VertexProperty(name: out_degree, type: integer, graph: sub-graph_23)

In [54]:
result = analyst.pagerank(graph)
result

VertexProperty(name: pagerank, type: double, graph: sub-graph_23)

In [73]:
graph.destroy_vertex_property_if_exists("approx_betweenness")
result = analyst.approximate_vertex_betweenness_centrality(graph, 100)
result

VertexProperty(name: approx_betweenness, type: double, graph: sub-graph_23)

In [89]:
result = analyst.eigenvector_centrality(graph)
result

VertexProperty(name: eigenvector, type: double, graph: sub-graph_23)

In [91]:
graph.query_pgql("""
  SELECT
    ID(a) AS id
  , a.in_degree
  , a.out_degree
  , a.pagerank
  , a.approx_betweenness
  , a.eigenvector
  FROM MATCH (a)
  WHERE LABEL(a) = 'ACCOUNT'
  LIMIT 10
""").print()

+--------------------------------------------------------------------------------------------------+
| id | in_degree | out_degree | pagerank              | approx_betweenness | eigenvector           |
+--------------------------------------------------------------------------------------------------+
| 0  | 11        | 40         | 3.5983441791575393E-5 | 104.49709350562382 | 1.2975692036835016E-4 |
| 1  | 12        | 10         | 3.575203057519747E-5  | 65.7718232584681   | 3.849965117169405E-5  |
| 2  | 5         | 20         | 2.638818724149162E-5  | 23.303157494654254 | 4.7213043338719186E-5 |
| 3  | 11        | 50         | 3.969588867386258E-5  | 175.58422092758997 | 1.6627449674103638E-4 |
| 4  | 5         | 20         | 2.451180169773911E-5  | 52.08889414019682  | 6.516924150293993E-5  |
| 5  | 8         | 50         | 3.1781903549304766E-5 | 40.018729935318156 | 1.347778467041246E-4  |
| 6  | 13        | 40         | 4.433780298702424E-5  | 69.05850389397104  | 1.405341942109

In [90]:
graph.query_pgql("""
  SELECT
    a.eigenvector,
    COUNT(*) AS count
  FROM MATCH (a)
  WHERE LABEL(a) = 'ACCOUNT'
  GROUP BY a.eigenvector
  ORDER BY count DESC
  LIMIT 10
""").print()

+-------------------------------+
| eigenvector           | count |
+-------------------------------+
| 6.373426281538385E-5  | 1     |
| 8.489945688799421E-5  | 1     |
| 8.93271202199486E-5   | 1     |
| 7.453045363561783E-5  | 1     |
| 1.1769880823077846E-4 | 1     |
| 1.4242949020520104E-4 | 1     |
| 1.3940538678912094E-4 | 1     |
| 1.7427731399268602E-4 | 1     |
| 2.2822500546161515E-5 | 1     |
| 4.8889451844046E-5    | 1     |
+-------------------------------+



## Community Detection

In [23]:
graph = graph.filter(pgx.EdgeFilter("edge.label()='TRANSFERRED_TO'"))
graph

PgxGraph(name: sub-graph_16, v: 10000, e: 299240, directed: True, memory(Mb): 14)

In [79]:
result = analyst.communities_label_propagation(graph)
result

PgxPartition(graph: sub-graph_23, components: 2031)

In [82]:
result = analyst.communities_conductance_minimization(graph)
result

PgxPartition(graph: sub-graph_23, components: 508)

In [83]:
graph.query_pgql("""
  SELECT
    ID(a) AS id
  , a.label_propagation
  , a.conductance_minimization
  FROM MATCH (a)
  WHERE LABEL(a) = 'ACCOUNT'
  LIMIT 10
""").print()

+---------------------------------------------------+
| id | label_propagation | conductance_minimization |
+---------------------------------------------------+
| 0  | 0                 | 203                      |
| 1  | 1                 | 0                        |
| 2  | 2                 | 485                      |
| 3  | 3                 | 350                      |
| 4  | 4                 | 435                      |
| 5  | 5                 | 178                      |
| 6  | 6                 | 1                        |
| 7  | 7                 | 203                      |
| 8  | 8                 | 118                      |
| 9  | 9                 | 226                      |
+---------------------------------------------------+



In [84]:
graph.query_pgql("""
  SELECT
    a.label_propagation,
    COUNT(*) AS count
  FROM MATCH (a)
  WHERE LABEL(a) = 'ACCOUNT'
  GROUP BY a.label_propagation
  ORDER BY count DESC
  LIMIT 10
""").print()

+---------------------------+
| label_propagation | count |
+---------------------------+
| 13                | 37    |
| 539               | 31    |
| 1005              | 31    |
| 506               | 28    |
| 333               | 27    |
| 856               | 27    |
| 363               | 23    |
| 248               | 23    |
| 802               | 22    |
| 149               | 22    |
+---------------------------+



## Closeness to Known Fraud Accounts

In [127]:
graph.query_pgql("""
  SELECT COUNT(*)
  FROM MATCH (a)
  WHERE a.is_fraud = true
""").print()

+----------+
| COUNT(*) |
+----------+
| 1        |
+----------+



In [123]:
rs = graph.query_pgql("SELECT ID(a) FROM MATCH (a) WHERE a.is_fraud = true")
vertex = graph.get_vertex(rs.get_row(0))
graph.destroy_vertex_property_if_exists("ppr")
analyst.personalized_pagerank(graph, vertex, rank="fraud_closeness")

VertexProperty(name: fraud_closeness, type: double, graph: sub-graph_44)

In [128]:
graph.query_pgql("""
  SELECT
    a.acc_id
  , a.fraud_closeness
  FROM MATCH (a)
  WHERE LABEL(a) = 'ACCOUNT'
  ORDER BY a.acc_id
  LIMIT 5
""").print()

+--------------------------------+
| acc_id | fraud_closeness       |
+--------------------------------+
| 0      | 1.0421308187008823E-5 |
| 1      | 0.15092796582155435   |
| 2      | 0.01130217355096834   |
| 3      | 2.334016436902037E-6  |
| 4      | 0.006495951565058347  |
+--------------------------------+



## Patterns

In [158]:
graph.query_pgql("""
  SELECT a1.acc_id, count(*)
  FROM MATCH (a1)-[t1:transferred_to]->(a2)-[t2:transferred_to]->(a3)
     , MATCH (a3)-[t3:transferred_to]->(a4)-[t4:transferred_to]->(a1)
  WHERE ID(a1) != ID(a3) AND ID(a2) != ID(a4)
    AND t1.amount >= 500 AND t2.amount >= 500 AND t3.amount >= 500 AND t4.amount >= 500
    AND t1.datetime < t2.datetime AND t2.datetime < t3.datetime AND t3.datetime < t4.datetime
  GROUP BY a1.acc_id
  ORDER BY a1.acc_id
  LIMIT 5
""").print()

+-------------------+
| acc_id | count(*) |
+-------------------+
| 24     | 1        |
| 30     | 1        |
| 39     | 1        |
| 40     | 4        |
| 44     | 2        |
+-------------------+



In [163]:
#graph.destroy_vertex_property_if_exists("num_4_hops_cycles")
graph.create_vertex_property("integer", "num_4_hops_cycles")

VertexProperty(name: num_4_hops_cycles, type: integer, graph: sub-graph_44)

In [165]:
graph = graph.clone_and_execute_pgql("""
  UPDATE a1 SET (a1.num_4_hops_cycles = COUNT(*))
  FROM MATCH (a1)-[t1:transferred_to]->(a2)-[t2:transferred_to]->(a3)
     , MATCH (a3)-[t3:transferred_to]->(a4)-[t4:transferred_to]->(a1)
  WHERE ID(a1) != ID(a3) AND ID(a2) != ID(a4)
    AND t1.amount >= 500 AND t2.amount >= 500 AND t3.amount >= 500 AND t4.amount >= 500
    AND t1.datetime < t2.datetime AND t2.datetime < t3.datetime AND t3.datetime < t4.datetime
  GROUP BY a1
""")

## Store Features to Data Frame

In [166]:
rs = graph.query_pgql("""
  SELECT
    a.acc_id
  , a.in_degree
  , a.out_degree
  , a.pagerank
  , a.approx_betweenness
  , a.eigenvector
  , a.label_propagation
  , a.conductance_minimization
  , a.fraud_closeness
  , a.num_4_hops_cycles
  , CAST(a.is_fraud AS STRING)
  FROM MATCH (a)
  WHERE LABEL(a) = 'ACCOUNT'
  ORDER BY a.acc_id
  LIMIT 5
""")

In [167]:
import numpy as np
import pandas as pd
def pgql_to_pandas(pgql):
    num_rows = pgql.num_results
    print("Converted to PGQL result set")
    # Fetch all vectors into the local RAM session
    temp_result = list(pgql._result_set_util.toList(pgql._pgql_result_set,0,num_rows))
    print("Fetched all rows")
    # Convert the result into numpy arrays
    temp_result2 = np.array(list(map(list,temp_result)))
    print("Converted all rows to lists")
    # Create a pandas dataframe containing the vectors
    vectors= pd.DataFrame(temp_result2[:,1:], index=temp_result2[:,0],
    columns = [str(i) for i in range(1,len(temp_result2[0]),1)])
    # Sort the index of the created dataframe
    vectors_sorted = vectors.sort_index()
    return vectors_sorted

In [168]:
df = pgql_to_pandas(rs)

Converted to PGQL result set
Fetched all rows
Converted all rows to lists


In [169]:
df

,1,2,3,4,5,6,7,8,9,10
0,11,40,3.5983441791575393e-05,104.49709350562382,0.00012975692036835016,0,203,1.0421308187008823e-05,0,false
1,9,20,2.939306882121337e-05,12.016183264107733,6.706991688594956e-05,1163,1,0.15092796582155435,0,true
2,12,10,3.575203057519747e-05,65.7718232584681,3.849965117169405e-05,1,0,0.01130217355096834,0,false
3,3,20,1.876739704409634e-05,19.096772807819328,5.134107046485419e-05,724,203,2.334016436902037e-06,0,false
4,5,20,2.638818724149162e-05,23.303157494654254,4.7213043338719186e-05,2,485,0.006495951565058347,0,false


## Unsupervised Learning

## Supervised Learning

superviesed learning needs sufficient number of positive cases (e.g. fraud cases) in the training dataset. Since the alerted cases are (often manually) investigated, such decitions (e.g. suspicious cases or not) might be the input.